In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import math
import sys
import os

import random
from scipy.stats import rice

In [ ]:
"""
Questions :
Est-ce que le E dans la formule de perte est une esperance ou une entropie ? 
A la fin on fait 
Chaque pacient as X images par dossier est-ce que on considere ce dossier on doit 
le considerer comme un ensamble d'images et que notre bute final est de debruiter l'ensamble d'images oubien
on les considere comme des images separer ? (pour le moment je les considere comme des images separer)

"""

In [33]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
print(output)
output.backward()

tensor(2.2842, grad_fn=<MseLossBackward0>)


In [34]:
def format_data_numpy(noised,denoised):
    data = []
    for i in range(denoised.shape[0]):
        pair = np.stack((noised[i],denoised[i]),axis=0)
        data.append(pair)
        
    data = np.array(data)
    return data

In [35]:
def format_data(noised, denoised):
    assert noised.shape == denoised.shape, "Input tensors must have the same shape."
    data = torch.stack((noised, denoised), dim=1)
    return data

In [36]:
def separate_data(data):
    assert data.shape[1] == 2, "Input tensor must have as second dim = 2 shape[1]"
    noised, denoised = torch.split(data, 1,dim=1)
    noised = noised.squeeze(1)
    denoised = denoised.squeeze(1)
    return noised, denoised 

General parameters of models :

In [32]:
lr = 0.0002

## Generator :
The generator takes denoised images and returnes noised images

In [41]:
#Generateur
#nb_feat : c'est le nombre de features que notre modele va gere 
class Generator(nn.Module):
    def __init__(self, nb_feat):
        super(Generator, self).__init__()
        self.nb_feat = nb_feat
        self.main = nn.Sequential(
            nn.Linear(nb_feat,100),
            nn.Tanh(),
            nn.Linear(100,nb_feat),
            nn.Tanh()
    )

    def forward(self, x):
        return self.main(x)

## Denoiser

In [44]:
class Denoiser(nn.Module):
    def __init__(self, nb_feat):
        super(Denoiser, self).__init__()
        self.nb_feat = nb_feat
        self.main = nn.Sequential(
            nn.Linear(nb_feat,100),
            nn.Tanh(),
            nn.Linear(100,nb_feat),
            nn.Tanh()
    )

    def forward(self, x):
        return self.main(x)

## Discriminator 
Takes two images and has to say if this pair of images is a real one or if it's fake => it takes the stack of the 2 images

In [46]:
class Discriminator(nn.Module):
    def __init__(self,nb_feat):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(nb_feat,100),
            nn.Tanh(),
            nn.Linear(100,nb_feat),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

In [59]:
generator = Generator(384)
denoiser = Denoiser(384)
discriminator = Discriminator(384)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = generator.to(device)
discriminator = discriminator.to(device)

The loss function follows the entropie :

In [6]:
class FLoss(nn.Module):
    def __init__(self):
        super(FLoss, self).__init__()
        
    def forward(self, logits, labels):
        # Example: Custom loss combining standard GAN loss with an additional term
        if logits.shape != labels.shape:
            raise ValueError("Input and labels must have the same shape")
            
        sigmoid_logits = torch.sigmoid(logits) 
        loss = ( labels * torch.log(sigmoid_logits + 1e-12) - 
                  (1 - labels) * torch.log(1 - sigmoid_logits + 1e-12))

        return loss.mean()


Now that we have our loss function we want to initialise the optimizer of our models, that we are going to add in the train function

In [38]:
#optimizer_g = torch.optim.Adam(generator.parameters(), lr=lr)
#optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=lr)

Then we have the training loop : \
We know that our dataloader gives us the truth so all the labels are set to true/1

Note : my data contain the denoised image and the the noised one stack behind. Also we have 

In [ ]:
"""
In order to call the model we have to :
- get the data reformat 
- create the models
- 
Note the  dataloaders with the noised,denoised and labels 

"""


Note : \
with the use of `.detach()` you ensure that the gradients from the discriminator do not flow back to the generator during the backward pass.

We get our transformed data :

In [23]:
def bruitage_racien(image,b = 0,loc=0,scale=1):
    noise = rice.rvs(b, loc=loc, scale=scale, size=image.shape)
    noisy_image = np.clip(image+noise, 0, 255)
    
    return noisy_image

For my data I want to change the data I want to extract them all and noised them.

In [18]:
def extract_nii_data(source_directory ,typeOfdata = '2_t2_tse_sag'):
    all_images = []
    #we iterate in the main directory
    for name in os.listdir(source_directory):
        #iteration for each patient
        inner_directory = source_directory+"/"+name
        for filename in os.listdir(inner_directory):
            #extract only the type we want t1,t2 
            if typeOfdata in filename :
                print(filename)
                all_images.append(np.array((nib.load(inner_directory+"/"+filename)).dataobj))
    # Check if the list is not empty         
    if all_images:  
        concatenated_images = np.concatenate(all_images, axis=0)
    else:
        concatenated_images = np.array([])
    return concatenated_images

In [19]:
data = extract_nii_data("./test")
data.shape

2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii
2_t2_tse_sag_384.nii


(133, 384, 384)

In [22]:
"""
The data that we noise we know that are images
"""
def noise_all_data(fnoise,data):
    assert data.ndim == 3,"Wrong type of data"
    noised = []
    for im in data: 
        noised.append(fnoise(im))
    noised = np.array(noised)
    return noised   

In [24]:
noised = noise_all_data(bruitage_racien,data)
noised.shape

(133, 384, 384)

In [31]:
#plt.imshow(np.rot90(noised[0]), cmap='gray')

In [30]:
#plt.imshow(np.rot90(data[0]), cmap='gray')

In [38]:
denoised_torch = torch.tensor(data, dtype=torch.float32) 
noised_torch = torch.tensor(noised, dtype=torch.float32) 

data_torch = format_data(noised_torch,denoised_torch)

data_torch.shape

torch.Size([133, 2, 384, 384])

Note the labels will depend on the MODEL :

In [40]:
#data_tensor = torch.from_numpy(data).float()/255.0
dataset = TensorDataset(denoised_torch, noised_torch)

batch_size = 10 
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#veryfication loop
for batch_denoised, batch_noised in dataloader:
    print(f"denoised : {batch_denoised.shape}, noised: {batch_noised.shape}")

batch_denoised, batch_noised = next(iter(dataloader))
noised_image = batch_noised[0]
denoised_image = batch_denoised[0]

denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([10, 384, 384]), noised: torch.Size([10, 384, 384])
denoised : torch.Size([3, 384, 384]), 

In [71]:
def train(data_loader,num_epochs,g_model,d_model,r_model,nb_feat,alpha,n_critic,loss_fn):
    
    optimizer_g = torch.optim.Adam(g_model.parameters(), lr=lr)
    optimizer_d = torch.optim.Adam(d_model.parameters(), lr=lr)
    optimizer_r = torch.optim.Adam(r_model.parameters(), lr=lr)
    
    g_model = g_model.to(device)
    d_model = d_model.to(device)
    r_model = r_model.to(device)
    
    #while loss small engouh
    
    for e in range(num_epochs):
        print(f"epoch {e} done")
        for n in range(n_critic):
            print(f"critic {n} done")
            for i, (denoised,noised) in enumerate(data_loader):

                #place it on the machine 
                noised = noised.to(device)
                denoised = denoised.to(device)
                #def separate to noised and denoised
            

                #labels
                fake_labels = torch.zeros(noised.size(0), 1, device=device)
                real_labels = torch.ones(noised.size(0), 1, device=device)


                ########################
                #Discriminator Training# 
                ########################

                optimizer_d.zero_grad()

                #optimize the discriminator by giving the real images

                real_data = format_data(noised,denoised)
                real_output = d_model(real_data)

                #generate fake data (start by giving a random noise)
                z = torch.randn(batch_size, nb_feat,nb_feat, device=device)#TODO : fix the size depending of the dataloaders
                fake_data_noised = g_model(z)
                fake_data_denoised = r_model(noised)
                print(fake_data_noised.shape)
                print(fake_data_denoised.shape)

                #(x_hat,y) : fake real
                fake_real = format_data(fake_data_noised,denoised)
                fake_real_out = d_model(fake_real)

                #(x,y_hat) : real fake
                real_fake = format_data(noised,fake_data_denoised)
                real_fake_out = d_model(real_fake)

                #optimize the discriminator by giving the generated images

                d_loss_real = loss_fn(real_output,real_labels)
                d_loss_real_fake = loss_fn(real_fake_out,fake_labels)
                d_loss_fake_real = loss_fn(fake_real_out,fake_labels)

                d_loss = d_loss_real - (alpha)*d_loss_fake_real - (1-alpha)*d_loss_real_fake 
                print(d_loss)
                d_loss.backward()
                optimizer_d.step()

                # Print losses
                if i % 100 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], '
                      f'Discriminator Loss: {d_loss.item():.4f}')
                    
        for i, (denoised,noised) in enumerate(data_loader):

            #place it on the machine 
            noised = noised.to(device)
            denoised = denoised.to(device)

            #labels
            fake_labels = torch.zeros(noised.size(0), 1, device=device)
            real_labels = torch.ones(noised.size(0), 1, device=device)


            ####################
            # Update Generator # 
            ####################

            optimizer_g.zero_grad()


            fake_data_denoised = r_model(noised)
            #fake_data = fake_data_noised + fake_data_denoised
            real_fake = format_data(noised,fake_data_denoised)
            real_fake_out = d_model(real_fake)


            g_loss =  alpha*loss_fn(real_fake_out,fake_labels)
            g_loss.backward()
            optimizer_g.step()

            ###################
            # Update Denoizer # 
            ###################

            optimizer_r.zero_grad()

            fake_data_noised = g_model(denoised)
            fake_real = format_data(fake_data_noised,denoised)
            fake_real_out = d_model(fake_real)

            r_loss =(1-alpha)*loss_fn(fake_real_out,fake_labels)
            r_loss.backward()
            optimizer_r.step()
            
            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], '
                      f'R : Denoiser Loss: {r_loss.item():.4f}')
                

In [72]:
lr = 0.0002
loss_fn = torch.nn.BCEWithLogitsLoss()
train(dataloader,2,generator,discriminator,denoiser,384,0.5,2,loss_fn)

epoch 0 done
critic 0 done
torch.Size([10, 384, 384])
torch.Size([10, 384, 384])


ValueError: Target size (torch.Size([10, 1])) must be the same as input size (torch.Size([10, 2, 384, 384]))

Have to think about the labaling :
So easy for the denoiser and for the generator because we only have 1 input.For the discriminator we consider that if at least one is not the real one then the label is fake so 0 otherwise it's 1(true).